In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!unzip "gdrive/My Drive/shopee-product-detection-student.zip"

Archive:  gdrive/My Drive/shopee-product-detection-student.zip
file #1:  bad zipfile offset (lseek):  0


In [ ]:
img_size = 224

# Data augmentation only on training set not val set
datagen_aug = ImageDataGenerator(
         rescale=1./255,
         validation_split=0.06,
         horizontal_flip=True,  # randomly flip images
         vertical_flip=False
         )  

batchsize = 16

train_gen = datagen_aug.flow_from_directory(
    directory="./train/train/train/",
    subset = 'training',
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=batchsize,
    class_mode="categorical",
    shuffle=True,
)

train_gen2 = datagen_aug.flow_from_directory(
    directory="./train/train/train/",
    subset = 'training',
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=batchsize,
    class_mode="categorical",
    shuffle=True,
)

valid_gen = datagen_aug.flow_from_directory(
    directory="./train/train/train/",
    subset = 'validation',
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

train_generator = CutMixImageDataGenerator(
    generator1=train_gen,
    generator2=train_gen2,
    img_size=img_size,
    batch_size=batchsize,
)

model = tf.keras.Sequential()

base = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False,input_shape=(img_size, img_size,3))
for layer in base.layers[:135]:
    layer.trainable = False
model.add(base)
model.add(GlobalAveragePooling2D())
#model.add(Dense(1024,
#        activation='relu',
#        kernel_initializer='he_normal',
#        kernel_regularizer=regularizers.l2(1e-3),
#    ))
#model.add(Dropout(0.6))
model.add(Dense(42, activation='softmax'))

optim = tf.keras.optimizers.SGD(lr=0.05, nesterov=True) # Start to lower it when it plateaus

# Compile model with optimizer
model.compile(
    optimizer=optim,
    loss='categorical_crossentropy',
    metrics=['accuracy']

  )


In [ ]:
# Training
epochs = 5

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 3)
cb_checkpointer = ModelCheckpoint(filepath = './gdrive/My Drive/checkpt_latest.hdf5', 
                                  monitor = 'val_loss',
                                  save_best_only = True, mode = 'auto')

VALID_STEP = valid_gen.n//valid_gen.batch_size
TRAIN_STEP = train_gen.n//train_gen.batch_size

model.load_weights('./gdrive/My Drive/checkpt_promising.hdf5') 
history = model.fit(train_generator,
                              epochs = epochs, 
                              validation_data = valid_gen,
                              validation_steps = VALID_STEP,
                              steps_per_epoch = TRAIN_STEP, # depends on amount of training data
                              callbacks=[cb_checkpointer, cb_early_stopper])
model.load_weights('./gdrive/My Drive/checkpt_latest.hdf5')         

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
    directory="./test/test/test/",
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

STEP_SIZE_TEST = test_gen.n//test_gen.batch_size
test_generator.reset()

pred = model.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)
predicted_class_indices = np.argmax(pred,axis=1)

filenames=test_generator.filenames
submission_df=pd.DataFrame({"filename":filenames,
                      "category":predicted_class_indices})

submission_df["category"] = submission_df["category"].apply(lambda x: np.random.randint(10))  # 00 to 41 inclusive
submission_df["category"] = submission_df["category"].apply(lambda x: "{:02}".format(x))  # pad zeroes
submission_df.to_csv("submission.csv", index=False)

In [ ]:
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,300000); # useful thing to keep Colab running I guess